**Cell 1 — Load from Bronze**

In [0]:
bronze_df = spark.read.table("sales_db.bronze_sales_orders")
print("✅ Bronze loaded")

**Cell 2 — Flatten the nested ordered_products array**

In [0]:
from pyspark.sql.functions import explode, col, from_unixtime, nullif, lit

silver_df = (bronze_df
    .withColumn("product", explode(col("ordered_products")))
    .select(
        col("order_number"),
        col("customer_id"),
        col("customer_name"),
        col("number_of_line_items").cast("int"),
        from_unixtime(nullif(col("order_datetime"), lit("")).cast("long")).alias("order_datetime"),
        col("product.id").alias("product_id"),
        col("product.name").alias("product_name"),
        col("product.price").cast("double").alias("price"),
        col("product.qty").cast("int").alias("quantity"),
        col("product.curr").alias("currency")
    )
)
display(silver_df)

**Cell 3 — Add calculated column**

In [0]:
from pyspark.sql.functions import round as spark_round

silver_df = silver_df.withColumn(
    "line_total",
    spark_round(col("price") * col("quantity"), 2)
)
display(silver_df)

In [0]:
silver_df = (silver_df
    .dropna(subset=["order_number", "customer_id", "product_id", "price", "quantity"])
    .dropDuplicates(["order_number", "product_id"])
)
print(f"Clean row count: {silver_df.count():,}")

In [0]:
(silver_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("sales_db.silver_sales_orders")
)
print("✅ Silver table saved!")

In [0]:
%sql
SELECT * FROM sales_db.silver_sales_orders LIMIT 10